## Introduction

This is an overview of the first experiment run within the context of the TC39. Mistakes were made, but this document should help us find our footing.

## Participant JS proficiency

Participants were graded on their JS proficiency in two ways. First, they were asked how they viewed their own proficiency, and then they were asked to answer two questions to identify if they were readily aware of simple but significant ways in which the language worked. The first were two problems on the output of a console log. The second was a test to see if they understood different ways a function could be written that was equivalent. These were the tests:

```js
function add(x, y) {
  return x + y;
}

function addTo(x) { 
  return (y) => add(x, y);
} 

const addTen = addTo(10);

console.log(typeof add(1, 2));
console.log(typeof addTen);

/****Questions:
*
*    What is the result of the first console log?
*
*    What is the result of the second console log?
*    - A string
*    - An object
*    - An error
*    - A function
*    What is the type that is returned from `addTen(’15’)`?
*    - A number
*    - A number but with a value NaN
*    - A function
*    - A string
*
*    Which of these are equivalent to addTo(10)?
*
*    - add.bind(null, 10);
*    - add.call(null, 10);
*    - y => add(10, y);
*    - add.apply(null, [10]);
*
*****/

```

TODO: Number of correct or incorrect responses:

In [1]:
import pandas as pd 

data = pd.read_csv("numbers.csv") 


## Error insertion patterns

The two proposals were tested against one another in a few different ways. The proposals could be paired with either a "nested" variant of the code, or a "flattened" variant of the code. Participants were assigned code samples randomly and there were a total of 16 different code samples tested, focused on different aspects that each pattern would reveal. 

### Testing Existing Patterns against the new proposal patterns

The two tested proposals had very small differences in their syntax. I tried to establish the baseline between two very different looking existing patterns against the very similar new variant. However, I found that certain types of errors would really only happen in certain contexts. Pipeline 1, the pipeline with the `#`, would very rarely have an issue with a missing `)`. The similarity between the pipelines and the flattened variant ment that a missing `day` parameter in `ferment` would be minimal. However it could and should have been tested. None the less the results are interesting:


###  Missing Parentheses test

The missing parenthesis test tested if a parenthesis, which could be visually forgotten in a piece of code, was more easily picked up in one variant of the test as opposed to the others.
    
<table>
<tr><th style="text-align: left">Nested Variant</th>
    <th style="text-align: left">Flattened Variant</th>
    <th style="text-align: left">Proposal Variant 2</th>
</tr>
<tr >
<td style="text-align: left">
Bughunt 3:
    
```js
const sauerkrautJar = await ferment(
  salt(
    new Jar
      pound(
        cut(
          wash("cabbage")
        )
      )
    )
  ), 14
);
const sauerkraut = sauerkrautJar.takeOne();

// SyntaxError: unexpected token: ')'
```
</td><td style="text-align: left">
    
Bughunt 6:
```js
const washedCabbage = wash("cabbage");
const cutCabbage = cut(washedCabbage);
const crushedCabbage = poundcutCabbage);
const cabbageJar = new Jar(crushedCabbage);
const saltyCabbageJar = salt(cabbageJar);
const sauerkrautJar = await ferment(saltyCabbageJar, 14);
const sauerkraut = sauerkrautJar.takeOne();

// SyntaxError: unexpected token: ')'
```
</td><td style="text-align: left">
    
Bughunt 12:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> (_ => new Jar(_)
  |> salt
  |> (_ => ferment(_, 14))
  |> await
  |> (_ => _.takeOne())

// SyntaxError: missing ) in parenthetical
```
</td></tr>
</table>

In [ ]:
columns = ['Timing Correct Bughunt 3', 'Timing Incorrect Bughunt 3', 'Timing Correct Bughunt 6', 'Timing Incorrect Bughunt 6', 'Timing Correct Bughunt 12', 'Timing Incorrect Bughunt 12']
threshold_upper = 350
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

#### TODO: conclusion

On first glance, it appears that Proposal Variant 2 performed better here, significantly better than the nested variant.

###  Missing second argument

<table align=left>
<tr><th style="text-align: left">Nested Variant</th>
    <th style="text-align: left">Pipeline Variant 1</th>
    <th style="text-align: left">Pipeline Variant 2</th>
</tr>
<tr >
<td align=left style="text-align: left">

Bughunt 1:

```js
const sauerkrautJar = await ferment(
  salt(
    new Jar(
      pound(
        cut(
          wash("cabbage")
        )
      )
    )
  )
);
const sauerkraut = sauerkrautJar.takeOne();

// Error: No day length provided
```
</td><td align=left style="text-align: left">

Bughunt 10:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> new Jar(#)
  |> add("water", #)
  |> salt
  |> ferment
  |> await #
  |> #.takeOne()

// Error: No day length provided
```
</td><td align=left style="text-align: left">

Bughunt 16:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> (_ => new Jar(_))
  |> salt
  |> ferment
  |> await
  |> (_ => _.takeOne())


// Error: No day length provided
```
</td></tr></table>



In [ ]:
columns = ['Timing Correct Bughunt 1', 'Timing Incorrect Bughunt 1', 'Timing Correct Bughunt 10', 'Timing Incorrect Bughunt 10', 'Timing Correct Bughunt 16', 'Timing Incorrect Bughunt 16']
threshold_upper = 350
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

#### TODO: conclusion

On first glance, it appears that Proposal Variant 2 performed better here, significantly better than the nested variant, but very close to proposal 1

## Pipeline Shared Error patterns

The two pipeline patterns shared a couple of error spaces, specifically in the implicit passing of arguments. This was tricky in particular for `new` instantiation, which would not automatically be done for the user. This is inconsistant syntax wise, both for the existing language and for the new pipeline operator. This was identified in testing as a tricky area for users. 

The two pipeline operators performed in the following way:

###  Pipeline only: `New` Object initialization


<table align=left>
<tr><th style="text-align: left">Pipeline Variant 1</th>
    <th style="text-align: left">Pipeline Variant 2</th>
</tr>
<tr >
<td align=left style="text-align: left">

Bughunt 9:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> new Jar
  |> add("water", #)
  |> salt
  |> ferment(#, 14)
  |> await #
  |> #.takeOne()

// Error: Jar must be instantiated with an initial ingredient
```
</td><td align=left style="text-align: left">

Bughunt 15:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> new Jar
  |> salt
  |> (_ => ferment(_, 14))
  |> await
  |> (_ => _.takeOne())

// Error: Jar must be instantiated with an initial ingredient
```
</td></tr></table>

In [ ]:
columns = ['Timing Correct Bughunt 9', 'Timing Incorrect Bughunt 9', 'Timing Correct Bughunt 15', 'Timing Incorrect Bughunt 15']
threshold_upper = 350
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

#### TODO: conclusion

The two proposals faired almost equally here, with a slightly better performance in pipeline variant 2.

###  Pipeline only: Early Line termination

Early termination is another tricky area for users. If a `;` is found in the pipeline, it will result in the next `|>` to be seen as a syntax error. This is something that is very likely if a user copy pastes from another area in the code.

The two variants performed in the following ways:

<table align=left>
<tr><th style="text-align: left">Pipeline Variant 1</th>
    <th style="text-align: left">Pipeline Variant 2</th>
</tr>
<tr >
<td align=left style="text-align: left">

Bughunt 8:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> new Jar(#);
  |> add("water", #)
  |> salt
  |> ferment(#, 2)
  |> await #
  |> #.takeOne()

// SyntaxError: unexpected token `|`
```
</td><td align=left style="text-align: left">

Bughunt 13:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> (_ => new Jar(_));
  |> salt
  |> (_ => ferment(_, 14))
  |> await
  |> (_ => _.takeOne())

// SyntaxError: Unexpected token `|`
```
</td></tr></table>

In [ ]:
columns = ['Timing Correct Bughunt 8', 'Timing Incorrect Bughunt 8', 'Timing Correct Bughunt 13', 'Timing Incorrect Bughunt 13']
threshold_upper = 350
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

#### TODO: conclusion

The two proposals faired almost equally here, with a slightly better performance in pipeline variant 1.

## Pattern Specific errors

Each pattern presented specific errors that were more likely to appear in their context rather than in others. These errors cannot be compared, but they can give us some data about how difficult different tasks are, using the same problem space. I intend to use this to grade how difficult each test was from the perspective of users. 

###  Flattened Variant Specific: Hidden in text error

Goal to see how long it takes for this type of error to be recognized.

Bugunt 4:
```js
const washedCabbage = wash("cabbage");
const cutCabbage = cut(washedCabbage);
const crushedCabbage = pound(cutCabbage);
const cabbageJar = new Jar(crushedCabbage);
const saltyCabbageJar = salt(cabbageJar);
const sauerkrautJar = await ferment(saltCabbageJar, 14);
const sauerkraut = sauerkrautJar.takeOne();

// Error: saltCabbageJar is undefined
```


In [ ]:
columns = ['Timing Correct Bughunt 4', 'Timing Incorrect Bughunt 4']
threshold_upper = 550
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

###  Pipeline Variant 1: Missing `#`

Goal to see how long it takes for this type of error to be recognized in this unique problem area for this proposal.

Bughunt 11:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> new Jar(#)
  |> add("water", #)
  |> salt()
  |> ferment(#, 2)
  |> await #
  |> #.takeOne()

// SyntaxError: `#` is undefined
```


In [ ]:
columns = ['Timing Correct Bughunt 11', 'Timing Incorrect Bughunt 11']
threshold_upper = 550
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

###  Pipeline Variant 2: Incorrect use of automatic return statement

Goal to see how long it takes for this type of error to be recognized in this unique problem area for this proposal.

Bughunt 14:
```js
const sauerkraut = "cabbage"
  |> wash
  |> cut
  |> pound
  |> (_ => {new Jar(_)})
  |> salt
  |> (_ => {ferment(_, 14)})
  |> await
  |> (_ => {_.takeOne()})


// Error: jar is undefined
```

In [ ]:
columns = ['Timing Correct Bughunt 14', 'Timing Incorrect Bughunt 14']
threshold_upper = 550
threshold_lower = 10

for column in columns:
    data[column].dropna()
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)

boxplot = data.boxplot(column=columns, figsize=(20, 5))

# Comparing Typing

### typing performance and correctness in translating a given challenge

Participants who took the proposal 1, 2 or mixed branches were asked to refactor the following code:

```js
const kimchiJar = await ferment(
    stir(
      add(
        "spicy paste",
        new Jar(
          salt(
            cut(
              wash("cabbage")
            )
          )
        )
      )
    )
  ), 2
);

const kimchi = kimchiJar.takeOne();
```

The code itself had no errors. The correctness of the implementation was then marked by hand, and reviewed. Participants were timed in this task.

The results were as follows, for correct and incorrect implementations.


In [ ]:
columns = ['Proposal 1 Implementation Timing Correct', 'Proposal 1 Implementation Timing Incorrect', 'Proposal 2 Implementation Timing Correct','Proposal 2 Implementation Timing Incorrect']
threshold_upper = 1000
threshold_lower = 0

for column in columns:
    data[column] = data[column].clip(upper=threshold_upper, lower=threshold_lower)
    data[column].dropna()



boxplot = data.boxplot(column=columns, figsize=(20, 5))